* https://huggingface.co/blog/how-to-generate
* http://jalammar.github.io/illustrated-gpt2/#part-2-illustrated-self-attention

In [1]:
# !pip install -q git+https://github.com/huggingface/transformers.git

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ~/Documents/github/just-a-recipe-generator

/Users/EGK1/Documents/github/just-a-recipe-generator


In [3]:
import copy
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [4]:
from src.data.pickling import load_pickle
from src.features.clean_shoestring_data import clean_shoestring_recipes
from src.features.prepare_model_data import (ingredients_to_text, tokenize_text,
                                             create_sequences, split_input_target,
                                             make_training_data, text_from_ids)

[nltk_data] Downloading package punkt to /Users/EGK1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/EGK1/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [5]:
shoestring_recipes0 = load_pickle("data/raw/shoestring_recipes.pickle")

Loading data/raw/shoestring_recipes.pickle for consumption...


In [6]:
shoestring_recipes = copy.deepcopy(shoestring_recipes0)

In [7]:
recipe_dict = clean_shoestring_recipes(shoestring_recipes)

In [9]:
text = ingredients_to_text(recipe_dict)
# vocab, ids_from_chars, chars_from_ids = tokenize_text(text)
# sequences = create_sequences(text, ids_from_chars)
# dataset0 = sequences.map(split_input_target)
# dataset = make_training_data(dataset0)

In [13]:
text[:50]

'Ingredients:\n 5.0 ounces canned pumpkin puree\n 0.5'

## Encode context and generate text

In [10]:
# load and init model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# add the EOS token as PAD token to avoid warnings
model = TFGPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

Metal device set to: Apple M1


2022-10-11 14:23:55.020806: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-10-11 14:23:55.021409: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [14]:
# encode context the generation is conditioned on
input_ids = tokenizer.encode(text, return_tensors='tf', max_length=1024, truncation=True)

## Beam Search

A decoder method
* Reduces the risk of missing hidden high prob word sequences by keeping the most likely number of beams of hypotheses at each time step and choosing the hypothesis that has the overall highest probability. 
* Always finds output sequence with higher prob than greedy search
* Not guaranteed to find most likely output

In [16]:
# set return_num_sequences > 1
beam_outputs = model.generate(
    input_ids, 
    max_length=3987,
    num_beams=5, 
    no_repeat_ngram_size=2, 
    num_return_sequences=5, 
    early_stopping=True
)
